### LSTM model for POS tagging

In [1]:
import torch

In [5]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super().__init__()
        self.hidden_data = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, hidden_dim)
        
        #emedding to hidden state
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        #hidden state to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, target_size)


    def forward(self, x):
        embed = self.word_embeddings(x)
        lstm_out, _ = self.lstm(embed.view(len(x), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        tag_scores = torch.nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores





Prepare Data

In [3]:
def prepare_sequence(seq, word_to_ix):

    idxs = [word_to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
                    ###Tags: 'DET': Determiner, 'NN': Noun, 'V': Verb
                    ('The dog ate the pie'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
                    ('All hail the king'.split(), ['DET', 'V', 'DET', 'NN']),

                ]

#Assign each word a unique index
word_to_ix={}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

#unique index for each tag
tag_to_ix = {'DET':0, 'NN':1, 'V':2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'pie': 4, 'All': 5, 'hail': 6, 'king': 7}


Train the model

In [8]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-2)


In [9]:

#Score before training
#Score i,j is for word i and tag j
with torch.no_grad():
    output = model(prepare_sequence(training_data[0][0], word_to_ix))
    print(output)

tensor([[-0.8472, -1.2827, -1.2238],
        [-0.8954, -1.3289, -1.1185],
        [-0.9093, -1.3191, -1.1092],
        [-0.8804, -1.4042, -1.0794],
        [-0.8234, -1.4640, -1.1094]])


Scores after training

In [11]:
for epoch in range(100):
    for sentence, tags in training_data:

        #Step 1: Clear the Gradients
        model.zero_grad()

        #Step 2: Get inputs ready
        inputs = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        #Step 3: Run forward pass
        tag_scores = model(inputs)

        #Step 4: Compute loss, grads and update weights
        loss = loss_fn(tag_scores, targets)
        loss.backward()
        optimizer.step()




In [12]:
with torch.no_grad():

    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-0.8138, -1.1846, -1.3825],
        [-0.8666, -1.2047, -1.2735],
        [-0.8840, -1.1945, -1.2587],
        [-0.8391, -1.2528, -1.2652],
        [-0.7763, -1.3047, -1.3143]])


#### ToDo: Create Character Embeddings